#                                 Implementation of Unet using pytorch

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from PIL import Image
import time
from torch.utils.data import Dataset, DataLoader
import cv2
import glob
from tqdm import tqdm, tqdm_notebook



In [3]:
print("Is cuda available? \n  ", torch.cuda.is_available())
device = torch.device('cuda')


Is cuda available? 
   True


In [5]:
ls

 Volume in drive D has no label.
 Volume Serial Number is 729E-49C0

 Directory of d:\projects\Satelite-image-classifiaction-using-ML\unet_model

05/23/2022  05:06 PM    <DIR>          .
05/23/2022  04:02 PM    <DIR>          ..
05/23/2022  04:58 PM             1,180 requirements.txt
05/23/2022  04:58 PM             7,550 source.ipynb
05/23/2022  04:58 PM            19,483 source_ver1.ipynb
05/23/2022  04:58 PM    <DIR>          test
05/23/2022  04:58 PM    <DIR>          train
05/23/2022  04:58 PM        11,768,865 UNet Convolutional Networks for Biomedical Image Segmentation.pdf
               4 File(s)     11,797,078 bytes
               4 Dir(s)  1,978,551,009,280 bytes free


## Creating data path list

In [7]:
#aumented data
aug_data_folder = './train/aug*'
aug_input_filepaths = sorted(glob.glob(os.path.join(aug_data_folder,"image*")))
aug_target_filepaths = sorted(glob.glob(os.path.join(aug_data_folder,"image*")))
print(aug_input_filepaths[1])
print(aug_target_filepaths[1])

#original data
train_path = './train/image'
label_path = './train/label'

train_imgs = sorted(glob.glob(os.path.join(train_path,"*")))
labels =  sorted(glob.glob(os.path.join(label_path,"*")))
print(train_imgs[12])
print(labels[12])

#input_filepaths=aug_input_filepaths+train_imgs
#target_filepaths = aug_target_filepaths + labels

input_filepaths= train_imgs
target_filepaths =  labels

print("\n checking the final folder")
print(input_filepaths[12])
print(target_filepaths[12])


./train\aug\image_0_9312159.png
./train\aug\image_0_9312159.png
./train/image\2.png
./train/label\2.png

 checking the final folder
./train/image\2.png
./train/label\2.png


In [8]:
test_img = cv2.imread(input_filepaths[12],cv2.IMREAD_UNCHANGED)
new_shape = (test_img.shape[0],test_img.shape[1],1)
test_img = test_img[np.newaxis,:,:]
test_img.shape
print("size of one image on with float data type in mega bytes :",4*512*512/(1024*1024))

size of one image on with float data type in mega bytes : 1.0


## Creating the Dataset and Dataloader objects

In [9]:
#creating dataset and dataloader object

class unet_dataset(Dataset):
    def __init__(self,input_filepaths, target_filepaths):
        
        self.input_filepaths = input_filepaths
        self.target_filepaths = target_filepaths
        assert len(target_filepaths) == len(input_filepaths)
        
        
    def __len__(self):
        return len(input_filepaths)
    
    def __getitem__(self,idx):
        img = cv2.imread(self.input_filepaths[idx],cv2.IMREAD_UNCHANGED)
        img = img.astype(np.float32)/255
        img = img[np.newaxis,:,:]
        img = torch.from_numpy(img)
        
        target = cv2.imread(self.target_filepaths[idx],cv2.IMREAD_UNCHANGED)
        target = target.astype(np.int64)

        target = torch.from_numpy(target)
        return img, target
        
        

In [10]:
#testing

train_dataset = unet_dataset(input_filepaths,target_filepaths)
print(train_dataset.__len__())
batch_size = 1 
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True, num_workers=os.cpu_count())



30


In [22]:
progress_bar = tqdm_notebook(dataloader)

for i,j in enumerate(progress_bar):
    print(i)

C:\Users\Nikesh\AppData\Local\Temp\ipykernel_22988\2858341834.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progress_bar = tqdm_notebook(dataloader)


  0%|          | 0/30 [00:00<?, ?it/s]

RuntimeError: DataLoader worker (pid(s) 26844, 20860, 19240, 21248, 20968, 20800, 27580, 26780, 6112, 7668, 8636, 27192) exited unexpectedly

## Configuring the model 

In [12]:
"""
u net is a fullly convolutional network 

contracting path
convulution layer
relu layer
max pooling

expanding path
convolution layer
updampling layer


"""

class UNet(nn.Module):
    def __init__(self,in_channels=1,n_classes=2,depth=5,wf=6, padding=False):
        super(UNet,self).__init__()
        self.depth = depth
        self.down_path = nn.ModuleList()
        self.up_path = nn.ModuleList()

        prev_channels = in_channels

        for i in range(depth):
            self.down_path.append(UNetConvBlock(prev_channels, 2**(wf+i)))
            
            prev_channels = 2**(wf+i)
            
        for i in reversed(range(depth - 1)):
            self.up_path.append(UNetUpBlock(prev_channels, 2** (wf + i)))
            prev_channels = 2 ** (wf + i)

        self.last = nn.Conv2d(prev_channels, n_classes, kernel_size=1)        
        
    def forward(self,x):
        blocks = []
        for i, down in enumerate(self.down_path):
            x = down(x)
            if i != len(self.down_path)-1 :
                blocks.append(x)
                x = F.max_pool2d(x,2) 
            
        for i, up in enumerate(self.up_path):
            x = up(x, blocks[-i - 1])

        
        return self.last(x)
        
    

In [13]:

#building the model
  
class UNetConvBlock(nn.Module):
    def __init__(self,in_size, out_ch ):
        super(UNetConvBlock, self).__init__()

        block = []
        block.append(nn.Conv2d(in_size, out_ch,kernel_size=3))
        block.append(nn.ReLU())
        block.append(nn.Conv2d(out_ch,out_ch,kernel_size=3))
        block.append(nn.ReLU())
        
        self.block = nn.Sequential(*block)
        
        
    def forward(self,x):
        out = self.block(x)
        return out  

class UNetUpBlock(nn.Module):
    def __init__(self,in_size, out_size):
        super(UNetUpBlock,self).__init__()
        self.up = nn.ConvTranspose2d(in_size, out_size, kernel_size=2, stride=2)
        
        self.conv_block = UNetConvBlock(in_size, out_size)
        
    def center_crop(self, layer, target_size):
        _, _, layer_height, layer_width = layer.size()
        diff_y = (layer_height - target_size[0]) // 2
        diff_x = (layer_width - target_size[1]) // 2
        return layer[
            :, :, diff_y : (diff_y + target_size[0]), diff_x : (diff_x + target_size[1])
        ]

    def forward(self, x, bridge):
        up = self.up(x)
        crop1 = self.center_crop(bridge, up.shape[2:])
        out = torch.cat([up, crop1], 1)
        out = self.conv_block(out)

        return out

In [14]:
def get_unet_model(in_channels=1, num_output_classes=2,padding=False):
    model = UNet(in_channels=1, n_classes=num_output_classes, wf=6, depth=5, padding=True)
    
    
    # Optional, for multi GPU training and inference
    #model = nn.DataParallel(model)
    return model


In [15]:
model = get_unet_model(in_channels = 1,padding = False)
epochs = 1
optim = torch.optim.Adam(model.parameters(), lr=1e-3)

model = model.to(device)
for epoch in range(1, epochs+1):
    print("Epoch", epoch)
    
    epoch_losses = []
    progress_bar = tqdm_notebook(dataloader)
    
    for ii, (X, target) in enumerate(progress_bar):
        X = X.to(device)  # [N, 1, H, W]
        print(X.size())
        target = target.to(device)  # [N, H, W] with class indices (0, 1)
        
        prediction = model(X)  # [N, 2, H, W]
        
        loss = F.cross_entropy(prediction, target, weight=None)
        break
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        epoch_losses.append(loss.detach().cpu().numpy())


Epoch 1


C:\Users\Nikesh\AppData\Local\Temp\ipykernel_22988\3708569981.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progress_bar = tqdm_notebook(dataloader)


  0%|          | 0/30 [00:00<?, ?it/s]

RuntimeError: DataLoader worker (pid(s) 18600, 27340, 26108, 27196, 24480, 25784, 25592, 24600, 6844, 14168, 2736, 21512) exited unexpectedly

In [13]:
prediction.shape

torch.Size([1, 2, 324, 324])